In [14]:
#Load Model
#from keras.models import load_model
#import tensorflow as tf
#from tensorflow.keras.utils import to_categorical
import numpy as np
from music21 import converter, instrument, note, chord, stream, tempo

#model = load_model("/home/sparkbrains/Documents/weights.399.0.9669.hdf5")

In [5]:


notes=[]
#generate notes from sample file
midi = converter.parse('/home/cygnet/Documents/Gaurav/Codes/caitsith.mid')
notes_to_parse = None
parts = instrument.partitionByInstrument(midi)
if parts: # file has instrument parts
    notes_to_parse = parts.parts[0].recurse()
else: # file has notes in a flat structure
    notes_to_parse = midi.flat.notes
    print(note_to_parse)
for element in notes_to_parse:
    if isinstance(element, note.Note):
        notes.append(str(element.pitch))
    elif isinstance(element, chord.Chord):
        notes.append('.'.join(str(n) for n in element.normalOrder))



In [6]:
len(notes)

1000

In [7]:
# unique notes
n_vocab= len(set(notes))
print(n_vocab)
# get all pitch names
pitchnames = sorted(set(item for item in notes))
print(pitchnames)

64
['0.4.7', '1.2', '1.5.8', '10.2', '10.2.5', '10.3', '3.4.8.10', '3.5', '3.6.8', '3.7.10', '3.8', '5.10', '5.7.0', '5.7.10', '5.8', '5.8.0', '5.8.10', '6.10', '7.0', '7.11.2', '7.8.0', '8.0', '8.0.1', '8.0.3', '8.1', '8.10', '8.10.3', 'A2', 'B-2', 'B-4', 'B-5', 'B4', 'C#2', 'C#3', 'C#4', 'C#5', 'C#6', 'C2', 'C3', 'C4', 'C5', 'D2', 'D4', 'D5', 'E-2', 'E-3', 'E-4', 'E-5', 'E2', 'E4', 'E5', 'F#2', 'F#4', 'F#5', 'F2', 'F3', 'F4', 'F5', 'G#2', 'G#4', 'G#5', 'G2', 'G4', 'G5']


['F2',
 '7.8.0',
 'C2',
 '7.8.0',
 '5.8',
 'B4',
 '5.8',
 'B4',
 'C#2',
 'G#2',
 'B-4',
 'G#4',
 'E4',
 'B4',
 'F2',
 '7.8.0',
 'C2',
 '7.8.0',
 '5.8',
 'B4',
 'C#2',
 'C#2',
 'E-3',
 'C#3',
 'G#2',
 'C3',
 'B-2',
 'F#2',
 'C3',
 'F2',
 'C2',
 'C#2',
 'G#2',
 'F2',
 'C2',
 'C#2',
 'C#2',
 'C#2',
 'E-2',
 'E-2',
 'E2',
 'E2',
 'F#2',
 'F#2',
 'F2',
 'C2',
 'C#2',
 'G#2',
 '5.8',
 'B4',
 'E-5',
 'C#5',
 'G#4',
 'F2',
 'C5',
 'B-4',
 'F#4',
 'C5',
 'C2',
 'B4',
 '7.8.0',
 'C#2',
 '7.8.0',
 'A2',
 '5.8',
 'B4',
 'F#2',
 'E2',
 'F2',
 'C2',
 'C#2',
 'G#2',
 'F2',
 'C2',
 'F2',
 'C2',
 'F2',
 'C2',
 'C#2',
 'G#2',
 'F2',
 'E-2',
 'D2',
 'C#2',
 'C2',
 '5.8',
 'B4',
 'B-4',
 'G#4',
 'C#2',
 'E4',
 'B4',
 'C#2',
 '7.8.0',
 'C2',
 'G#2',
 '7.8.0',
 'F2',
 'E4',
 '8.10',
 'G2',
 'G#2',
 'G2',
 'F2',
 'C3',
 'F#2',
 'G2',
 'C2',
 'C2',
 'D2',
 'E2',
 'F2',
 'C2',
 'C#2',
 'G#2',
 'F2',
 'C2',
 'C#2',
 'C#2',
 'E4',
 '8.10',
 'E-3',
 'C#3',
 'G#2',
 'C3',
 'B-2',
 'F#2',
 'C3',
 'F2',
 'E4',
 'G#4

In [5]:

""" Prepare the sequences used by the Neural Network """
sequence_length = 50

note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
network_input = []
network_output = []
for i in range(0, len(notes) - sequence_length, 1):
    sequence_in = notes[i:i + sequence_length]
    sequence_out = notes[i + sequence_length]
    network_input.append([note_to_int[char] for char in sequence_in])
    network_output.append(note_to_int[sequence_out])
n_patterns = len(network_input)
network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
network_input = network_input / float(n_vocab)
network_output = to_categorical(network_output)


In [6]:
network_input.shape, network_output.shape

((950, 50, 1), (950, 64))

In [7]:
# Load int_to_note of model 
import pickle
with open('/home/sparkbrains/Documents/saved_int_to_note.pkl', 'rb') as f:
    int_to_note = pickle.load(f)


In [8]:
def generate_notes(model, network_input, pitchnames, n_vocab, i, int_to_note):
    """ Generate notes from the neural network based on a sequence of notes """
    start = np.random.randint(0, len(network_input)-1)

   

    pattern = network_input[start]
    prediction_output = []
    for note_index in range(i):
        
        prediction_input = np.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input 

        prediction = model.predict(prediction_input, verbose=0)

        index = np.argmax(prediction)
        
        if index == pattern[-1] and index== pattern[-2]: #and index== pattern[-3] and index== pattern[-4]:
            index = np.random.randint(0,n_vocab-1)
        
        result = int_to_note[index]
        prediction_output.append(result)

        pattern = np.append(pattern, index)
        pattern = pattern[1:len(pattern)]

    return prediction_output


In [9]:
#Variable length output100
variable_lenght_output = int(input())
prediction_output= generate_notes(model, network_input, pitchnames, n_vocab, variable_lenght_output , int_to_note)

200


In [10]:
len(prediction_output)

200

In [ ]:
{1:Instrument.Flute()}

In [44]:
#convert the output from the prediction to notes and create a midi file from the notes 
# Create melody of multi intsruemnt 
Instruments= ["AcousticBass()","Piano()", "AcousticGuitar()", "ElectricPiano()", "Guitar()", "Flute()", "Violin()", 
    "Trumpet()","StringInstrument()","BassDrum()"]
offset = 0
output_notes = []
# create note and chord objects based on the values generated by the model
for pattern in notes:
    # pattern is a chord
    if ('.' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('.')
        notes = []
        for current_note in notes_in_chord:
#             method = f"instrument.{Instruments[6]}"

#             output_notes.append(method)
            output_notes.append(instrument.BassDrum())
            cn=int(current_note) 
            new_note=note.Note(cn, quarterLength=0.5)
            notes.append(new_note)
        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)   
            
      
    # pattern is a note
    else:
        output_notes.append(instrument.BassDrum())
#         method = f"instrument.{Instruments[6]}"

#         output_notes.append(method)
        new_note = note.Note(pattern,quarterLength=0.5)
        new_note.offset = offset
        
        output_notes.append(new_note)

# change and increase ending melody
notes=[]
for i , j in enumerate(output_notes):
    try:
        notes.append(output_notes[i].name)
        
    except:
        pass    
midi_stream = stream.Stream()
midi_stream.insert(0, tempo.MetronomeMark(number=500))

midi_stream.append(output_notes)
name2 = notes[-1]
for i in range(2):
    midi_stream.append(note.Note(name2,type = 'half'))

midi_stream.write('midi', fp='try6.mid')



'try6.mid'

In [33]:
a = stream.Stream()
a
a = a.insert(0, tempo.MetronomeMark(number=120))
a

In [30]:
midi_stream.write()

504.0

In [15]:
a = tempo.MetronomeMark('slow', 40, note.Note(type='half'))
a.number

40

In [17]:
output_notes[0].tempo.MetronomeMark('slow', 40, note.Note(type='half'))

AttributeError: 'BassDrum' object has no attribute 'tempo'

In [13]:
notes

['B',
 'E',
 'F#',
 'F',
 'G',
 'C',
 'A',
 'A',
 'D',
 'B-',
 'C',
 'C',
 'C',
 'C',
 'E',
 'B',
 'B',
 'C',
 'F',
 'C',
 'B-',
 'F#',
 'C',
 'C',
 'A',
 'D',
 'A',
 'D',
 'B',
 'B',
 'A',
 'G',
 'E',
 'E',
 'D',
 'G',
 'C',
 'E',
 'E',
 'C',
 'C',
 'C',
 'F',
 'A',
 'G',
 'C',
 'A',
 'F#',
 'G',
 'G',
 'D',
 'B-',
 'B-',
 'C',
 'A',
 'E-',
 'C',
 'F',
 'D',
 'B-',
 'D',
 'C',
 'E-',
 'F',
 'C',
 'B-',
 'E',
 'B',
 'A',
 'B',
 'A',
 'F',
 'B-',
 'C',
 'B-',
 'E-',
 'F#',
 'C',
 'C',
 'D',
 'F',
 'C',
 'A',
 'B-',
 'E',
 'C',
 'A',
 'D',
 'F#',
 'C#',
 'B-',
 'E-',
 'G',
 'E-',
 'C',
 'F',
 'B',
 'B-',
 'C',
 'E-',
 'B',
 'C',
 'A',
 'E',
 'G#',
 'B-',
 'B',
 'C',
 'F',
 'B',
 'E-',
 'G',
 'F',
 'B-',
 'G',
 'E',
 'B-',
 'C',
 'G',
 'D',
 'E',
 'A',
 'A',
 'E-',
 'G',
 'D',
 'E-',
 'B',
 'G',
 'C',
 'A',
 'E-',
 'E-',
 'G',
 'C',
 'C',
 'E-',
 'E',
 'E-',
 'C',
 'F',
 'C',
 'D',
 'A',
 'A',
 'E-',
 'F',
 'C',
 'F',
 'B-',
 'B-',
 'B-']

In [29]:


notes=[]
#generate notes from sample file
midi = converter.parse('/home/sparkbrains/Documents/Alphonso.mid')
notes_to_parse = None
parts = instrument.partitionByInstrument(midi)
if parts: # file has instrument parts
    notes_to_parse = parts.parts[0].recurse()
else: # file has notes in a flat structure
    notes_to_parse = midi.flat.notes
    print(note_to_parse)
for element in notes_to_parse:
    if isinstance(element, note.Note):
        notes.append(str(element.pitch))
    elif isinstance(element, chord.Chord):
        notes.append('.'.join(str(n) for n in element.normalOrder))

In [30]:
notes

['C5',
 'C5',
 'C5',
 'C5',
 'C5',
 'D5',
 'E5',
 'C5',
 'D5',
 '2',
 '2',
 'E5',
 'F5',
 'G5',
 'F5',
 'E5',
 'E5',
 'C5',
 'E5',
 'D5',
 'G4',
 'F#4',
 'B4',
 'G#4',
 'F4',
 'B-4',
 'C#4',
 'C4',
 'F4',
 'F4',
 'E-4',
 'G#4',
 'G4',
 'F4',
 'C#5',
 'C5',
 'F5',
 'E5',
 'G5',
 'A5',
 'F5',
 'F5',
 'B-5',
 'F#5',
 'C#5',
 'C#5',
 'C5',
 'B4',
 'G4',
 'G4',
 'D4',
 'C#4',
 'E4',
 'D4',
 'B-3',
 'G4',
 'C4',
 'B-4',
 'G#4',
 'G4',
 'G4',
 'G4',
 'G4',
 'G4',
 'A4',
 'B4',
 'G4',
 'A4',
 '9',
 '9',
 'B4',
 'C5',
 'D5',
 'E-5',
 'D5',
 'B-4',
 'F4',
 'C5',
 'G#4',
 'E-4',
 'B-3',
 'C4',
 'D4',
 'E-4',
 'F4',
 'E4',
 'F#4',
 'G#4',
 'B-4',
 'C5',
 'E4',
 'B-4',
 'D4',
 'G4',
 'B3',
 'B-3',
 'C#4',
 'E4',
 'F4',
 'E-5',
 'D5',
 'C#5',
 'D4',
 'B4',
 'B-4',
 'A4',
 'C5',
 'B-4',
 'F#4',
 'F4',
 'D4',
 'F#4',
 'E-4',
 'C4',
 'B3',
 'D4',
 'C4',
 'F4',
 'E4',
 'D4',
 'G4',
 'C5',
 'C5',
 '0',
 'C5',
 'C5',
 'D5',
 'E5',
 'C5',
 'D5',
 '2',
 '2',
 'E5',
 'F5',
 'G5',
 'F5',
 'E5',
 'E5',
 'C5',


In [17]:
import pretty_midi
pm = pretty_midi.PrettyMIDI('/home/sparkbrains/Documents/caitsith.mid')
print(pm.get_tempo_changes())

(array([0.]), array([126.000126]))


In [15]:
import midi
pattern = midi.read_midifile("/home/sparkbrains/Documents/caitsith.mid")

for track in pattern:
    for event in track:
        if isinstance(event, midi.NoteEvent): # check that the current event is a NoteEvent, otherwise it won't have the method get_pitch() and we'll get an error
            print(event.get_pitch())

ModuleNotFoundError: No module named 'midi'

In [5]:

li1= ["C5", "C#4", "D4", "E-3", "E5", "F5", "F#2", "G2", "G#2", "B-2", "B4", "A4"]
li2= ["C8", "C#7", "D8", "E-6", "E9", "F4", "F#10", "G5", "G#5", "B-4", "B8", "A8"]

In [6]:
for i in notes:
    if i in li1:
#     try:
#         if notes[i]== li1[i]:
#             notes[i].replace(li1[i], li2[j])
#     except:   
#         pass
# print(notes)

In [20]:
import music21.note


n = music21.note.Note("")
n.name

'D'

In [30]:
n = note.Note("C2")

n.pitch.frequency

65.40639132514957

In [58]:
# # change pitch of output
# for i , note in enumerate(output_notes):
#     try:
#         if output_notes[i].name == "D":
#             output_notes[i].name = 'A'
#     except:
#         pass

In [59]:
note.Note(2)

<music21.note.Note D>

In [174]:
midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp='m4.mid')

'm4.mid'

In [297]:
import music21
from music21 import converter, instrument, note, chord
file = '/home/sparkbrains/Documents/small.mid'
midi = converter.parse(file)  #read file
parts = instrument.partitionByInstrument(midi)

#print(parts.parts[0].getInstrument()
value_list = list()
for i in parts:
    ins_1 = i.getInstrument()
    value_list.append(ins_1)    
print(value_list)


    


[<music21.instrument.Flute 'Flute'>]


In [298]:
for j in value_list:
    print(j)

Flute


In [123]:
import music21
from music21 import converter, instrument, note, chord
file = '/home/sparkbrains/Documents/caitsith.mid'
midi = converter.parse(file)  #read file
parts = instrument.partitionByInstrument()
                                         
for i in midi.elements:
    print(i.                                   

TypeError: partitionByInstrument() missing 1 required positional argument: 'streamObj'

In [92]:
# intsruments names 
instrument_names = converter.parse("/home/sparkbrains/Documents/25296_110.mid")
print(instrument_names)
for m in instrument_names.par
    print(m[0])

<music21.stream.Score 0x7f3d8ffa9b10>
<music21.stream.Measure 1 offset=0.0>
<music21.stream.Measure 1 offset=0.0>
<music21.stream.Measure 1 offset=0.0>
<music21.stream.Measure 1 offset=0.0>
<music21.stream.Measure 1 offset=0.0>


In [ ]:
li=["AcousticBass","Piano", "AcousticGuitar", "ElectricPiano", "Guitar", "Flute", "Violin", "Trumpet","StringInstrument","BassDrum"]

In [349]:
file = '/home/sparkbrains/Documents/small.mid'
midi = converter.parse(file) 
inst=[]
def list_instruments(midi):
    partStream = midi.parts.stream()
    
    print("List of instruments found on MIDI file:")
    for p in partStream:
        print(p.partName)
        inst.append(p.partName)
        
    return inst

inst= list_instruments(midi)
print(inst)

List of instruments found on MIDI file:
Flute
['Flute']


In [76]:
import pretty_midi
file = pretty_midi.PrettyMIDI("/home/sparkbrains/Documents/small.mid")
print('Number of instruments:', len(file.instruments))
print('Instrument name:', pretty_midi.program_to_instrument_name(file.instruments[0].program))


Number of instruments: 2
Instrument name: Acoustic Grand Piano


In [79]:
# number of notes in  a particular instruments , total 5 instruments
instrument = file.instruments[1]
len(instrument.notes)

272

In [80]:
for i, note in enumerate(instrument.notes):
    print(f'{i}: pitch={note.pitch}, note_name={pretty_midi.note_number_to_name(note.pitch)},'
        f' duration={note.end - note.start:.4f}')

0: pitch=62, note_name=D4, duration=0.5000
1: pitch=65, note_name=F4, duration=0.5000
2: pitch=69, note_name=A4, duration=0.5000
3: pitch=71, note_name=B4, duration=0.5000
4: pitch=60, note_name=C4, duration=0.5000
5: pitch=55, note_name=G3, duration=0.5000
6: pitch=78, note_name=F#5, duration=0.5000
7: pitch=43, note_name=G2, duration=0.5000
8: pitch=83, note_name=B5, duration=0.5000
9: pitch=43, note_name=G2, duration=0.5000
10: pitch=67, note_name=G4, duration=0.5000
11: pitch=71, note_name=B4, duration=0.5000
12: pitch=62, note_name=D4, duration=0.5000
13: pitch=84, note_name=C6, duration=0.5000
14: pitch=79, note_name=G5, duration=0.5000
15: pitch=81, note_name=A5, duration=0.5000
16: pitch=72, note_name=C5, duration=0.5000
17: pitch=81, note_name=A5, duration=0.5000
18: pitch=48, note_name=C3, duration=0.5000
19: pitch=43, note_name=G2, duration=0.5000
20: pitch=60, note_name=C4, duration=0.5000
21: pitch=74, note_name=D5, duration=0.5000
22: pitch=77, note_name=F5, duration=0.50

In [92]:
f = note.Note("F5")

In [93]:
f.name

'F'

In [94]:
f.pitch

<music21.pitch.Pitch F5>

In [95]:
f.pitch.frequency

698.456462866008

In [341]:
from music21 import *
p1 = pitch.Pitch('B-')

In [342]:
p1.pitchClass

10

In [3]:
from music21 import converter, instrument, note, chord, stream

In [8]:
a= instrument.Piano()

In [9]:
a

<music21.instrument.Piano 'Piano'>